# Exercise 4

In [1]:
import numpy as np
import plotly.graph_objects as go
from tqdm import tqdm

In [2]:
r = 0.05
sigma = 0.2
S0 = 100
K = 100
T = 1.0

discount_factor = np.exp(-r * T)

In [3]:
N_paths_list = [100, 500, 1000, 5000, 10000, 50000, 100000]
N_steps_list_log = [1, 10, 100]

In [4]:
fig = go.Figure()

for N_steps in N_steps_list_log:
    estimates = []
    for N_paths in N_paths_list:
        dt = T / N_steps
        drift = (r - 0.5 * sigma**2) * dt
        diffusion = sigma * np.sqrt(dt)

        X = np.full(N_paths, np.log(S0))
        for _ in range(N_steps):
            dW = np.random.randn(N_paths)
            X += drift + diffusion * dW

        S_T = np.exp(X)
        payoff = np.maximum(S_T - K, 0)
        price_estimate = discount_factor * np.mean(payoff)
        estimates.append(price_estimate)

    fig.add_trace(go.Scatter(
        x=N_paths_list,
        y=estimates,
        mode='lines+markers',
        name=f"{N_steps} time steps"
    ))

fig.update_layout(
    title="Log-space Euler",
    xaxis=dict(
        title="N Paths",
        type="log"
    ),
    yaxis_title="Call Price",
    template="plotly_white",
    height=500
)

fig.show()

## Throughout the multiple runs of the above block, there appear to be no clear distinction amongst all the time intervals. Personally, I would choose the 1 time subinterval for computational cost.

## Reference Value

In [5]:
N_paths_ref = 200000
N_steps_ref = 1000

dt_ref = T / N_steps_ref
drift_ref = (r - 0.5 * sigma**2) * dt_ref
diffusion_ref = sigma * np.sqrt(dt_ref)

X_ref = np.full(N_paths_ref, np.log(S0))
for _ in range(N_steps_ref):
    dW = np.random.randn(N_paths_ref)
    X_ref += drift_ref + diffusion_ref * dW

S_T_ref = np.exp(X_ref)
payoff_ref = np.maximum(S_T_ref - K, 0)
reference_value = discount_factor * np.mean(payoff_ref)

## Simulate Convergence in original space

In [6]:
N_steps_list_original = [1, 2, 5, 10, 20, 50, 100, 200, 500, 1000]
N_paths_original = 100000

estimates_original = []

for N_steps in tqdm(N_steps_list_original):
    dt = T / N_steps

    S = np.full(N_paths_original, S0)
    for _ in range(N_steps):
        dW = np.random.randn(N_paths_original)
        S = S * np.exp((r - 0.5 * sigma**2) * dt + sigma * np.sqrt(dt) * dW)

    payoff = np.maximum(S - K, 0)
    price_estimate = discount_factor * np.mean(payoff)
    estimates_original.append(price_estimate)

fig2 = go.Figure()

fig2.add_trace(go.Scatter(
    x=N_steps_list_original,
    y=estimates_original,
    mode='lines+markers',
    name="Euler in S"
))

fig2.add_trace(go.Scatter(
    x=N_steps_list_original,
    y=[reference_value]*len(N_steps_list_original),
    mode='lines',
    name="Reference",
    line=dict(dash='dash')
))

fig2.update_layout(
    title="Original-space Euler",
    xaxis=dict(
        title="N Steps",
        type="log"
    ),
    yaxis_title="Call Price",
    template="plotly_white",
    height=500
)

fig2.show()

100%|██████████| 10/10 [00:08<00:00,  1.17it/s]


## The results plotted are different each time the block is executed. Then the sitmulation is done on original S_t instead of ln S_t, and with Euler discretization, the discretization introduces bias. This happens even with high discretization time intervals

# Exercise 5

In [7]:
r = 0.05
T = 0.5
discount_factor = np.exp(-r * T)

In [8]:
N_paths = 100000
sigmas = [0.1, 0.2, 0.4]
strike_prices = [90, 100, 110]

In [9]:
estimates_plain = []
variances_plain = []

estimates_anti = []
variances_anti = []

In [10]:
for sigma in sigmas:
    for K in strike_prices:
        S0 = 100

        drift = (r - 0.5 * sigma**2) * T
        diffusion = sigma * np.sqrt(T)

        # Plain
        Z = np.random.randn(N_paths)
        ST_plain = S0 * np.exp(drift + diffusion * Z)
        payoff_plain = np.maximum(ST_plain - K, 0) + np.maximum(K - ST_plain, 0)
        discounted_payoff_plain = discount_factor * payoff_plain

        mean_plain = np.mean(discounted_payoff_plain)
        var_plain = np.var(discounted_payoff_plain)

        estimates_plain.append(mean_plain)
        variances_plain.append(var_plain)

        # Antithetic
        Z = np.random.randn(N_paths)
        Z_antithetic = -Z

        ST_pos = S0 * np.exp(drift + diffusion * Z)
        ST_neg = S0 * np.exp(drift + diffusion * Z_antithetic)

        payoff_pos = np.maximum(ST_pos - K, 0) + np.maximum(K - ST_pos, 0)
        payoff_neg = np.maximum(ST_neg - K, 0) + np.maximum(K - ST_neg, 0)

        payoff_avg = 0.5 * (payoff_pos + payoff_neg)
        discounted_payoff_anti = discount_factor * payoff_avg

        mean_anti = np.mean(discounted_payoff_anti)
        var_anti = np.var(discounted_payoff_anti)

        estimates_anti.append(mean_anti)
        variances_anti.append(var_anti)

In [11]:
labels = []
for sigma in sigmas:
    for K in strike_prices:
        labels.append(f"σ={int(sigma*100)}%, K={K}")

fig = go.Figure()

fig.add_trace(go.Bar(
    x=labels,
    y=variances_plain,
    name="Plain Monte Carlo"
))

fig.add_trace(go.Bar(
    x=labels,
    y=variances_anti,
    name="Antithetic Variates"
))

fig.update_layout(
    title="Variance Comparison: Plain vs Antithetic",
    barmode='group',
    xaxis_title="Scenario (Volatility and Strike)",
    yaxis_title="Variance of Discounted Payoff",
    template="plotly_white",
    height=600
)

fig.show()

## The result indicates that antithetic consistently reduces the variance of estimated straddle price compared to plain Monte Carlo. The reduction is more significant at lower volatility fractionally, and higher volatility in absolute scale. The reduction is also more significant when the straddle is in the money. When the straddle gets close to at the money, the variance reduction becomes smaller, but still significant.

# Exercise 7

In [12]:
from sklearn.linear_model import LinearRegression

In [13]:
S0 = 100
K = 100
T = 5
r = 0.05
sigma = 0.2
n_simulations = 10000
n_steps = 10
dt = T / n_steps

times = np.linspace(0, T, n_steps + 1)

S = np.zeros((n_simulations, n_steps + 1))
S[:, 0] = S0

## Lognormal 5 Years

In [14]:
for t in range(1, n_steps + 1):
    Z = np.random.normal(0, 1, n_simulations)
    S[:, t] = S[:, t-1] * np.exp((r - 0.5 * sigma ** 2) * dt + sigma * np.sqrt(dt) * Z)

## Arithmetic and Geometric Asian; European

In [15]:
arithmetic_average = np.mean(S[:, 1:], axis=1)
geometric_average = np.exp(np.mean(np.log(S[:, 1:]), axis=1))

payoff_arith = np.exp(-r * T) * np.maximum(arithmetic_average - K, 0)
payoff_geom_control = np.exp(-r * T) * np.maximum(geometric_average - K, 0)
payoff_euro_control = np.exp(-r * T) * np.maximum(S[:, -1] - K, 0)

In [16]:
S_indep = np.zeros((n_simulations, n_steps + 1))
S_indep[:, 0] = S0

for t in range(1, n_steps + 1):
    Z = np.random.normal(0, 1, n_simulations)
    S_indep[:, t] = S_indep[:, t-1] * np.exp((r - 0.5 * sigma ** 2) * dt + sigma * np.sqrt(dt) * Z)

arithmetic_average_indep = np.mean(S_indep[:, 1:], axis=1)
geometric_average_indep = np.exp(np.mean(np.log(S_indep[:, 1:]), axis=1))

payoff_arith_indep = np.exp(-r * T) * np.maximum(arithmetic_average_indep - K, 0)
payoff_geom_indep = np.exp(-r * T) * np.maximum(geometric_average_indep - K, 0)
payoff_euro_indep = np.exp(-r * T) * np.maximum(S_indep[:, -1] - K, 0)

## Beta

In [17]:
cov_geom = np.cov(payoff_arith_indep, payoff_geom_indep)[0, 1]
var_geom = np.var(payoff_geom_indep)

beta_geom = cov_geom / var_geom

cov_euro = np.cov(payoff_arith_indep, payoff_euro_indep)[0, 1]
var_euro = np.var(payoff_euro_indep)

beta_euro = cov_euro / var_euro

In [18]:
control_geom = payoff_arith - beta_geom * (payoff_geom_control - np.mean(payoff_geom_control))
control_euro = payoff_arith - beta_euro * (payoff_euro_control - np.mean(payoff_euro_control))

## Variance Reduction: Actual vs Theoretical

In [19]:
var_original = np.var(payoff_arith)
var_control_geom = np.var(control_geom)
var_control_euro = np.var(control_euro)

variance_reduction_geom = var_original / var_control_geom
variance_reduction_euro = var_original / var_control_euro

theoretical_reduction_geom = 1 / (1 - (cov_geom**2) / (var_geom * np.var(payoff_arith_indep)))
theoretical_reduction_euro = 1 / (1 - (cov_euro**2) / (var_euro * np.var(payoff_arith_indep)))

In [20]:
reg_geom = LinearRegression().fit(payoff_geom_control.reshape(-1,1), payoff_arith)
line_geom = reg_geom.predict(payoff_geom_control.reshape(-1,1))

fig1 = go.Figure()

fig1.add_trace(go.Scatter(
    x=payoff_geom_control,
    y=payoff_arith,
    mode='markers',
    name='Data'
))

fig1.add_trace(go.Scatter(
    x=payoff_geom_control,
    y=line_geom,
    mode='lines',
    name='Best Fit Line'
))

fig1.update_layout(
    title=f"Arithmetic Asian vs Geometric Asian (β ≈ {beta_geom:.4f})",
    xaxis_title="Geometric Asian Payoff",
    yaxis_title="Arithmetic Asian Payoff",
    template="plotly_white"
)

fig1.show()

In [21]:
reg_euro = LinearRegression().fit(payoff_euro_control.reshape(-1,1), payoff_arith)
line_euro = reg_euro.predict(payoff_euro_control.reshape(-1,1))

fig2 = go.Figure()

fig2.add_trace(go.Scatter(
    x=payoff_euro_control,
    y=payoff_arith,
    mode='markers',
    name='Data'
))

fig2.add_trace(go.Scatter(
    x=payoff_euro_control,
    y=line_euro,
    mode='lines',
    name='Best Fit Line'
))

fig2.update_layout(
    title=f"Arithmetic Asian vs European Call (β ≈ {beta_euro:.4f})",
    xaxis_title="European Call Payoff",
    yaxis_title="Arithmetic Asian Payoff",
    template="plotly_white"
)

fig2.show()

In [22]:
print("Regression beta (geom):", round(reg_geom.coef_[0], 4))
print("Estimated beta (geom):", round(beta_geom, 4))

print("Regression beta (euro):", round(reg_euro.coef_[0], 4))
print("Estimated beta (euro):", round(beta_euro, 4))

Regression beta (geom): 1.088
Estimated beta (geom): 1.0835
Regression beta (euro): 0.4714
Estimated beta (euro): 0.4692


## The actual observed beta follows the theoretical values perfectly